In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os 

import seaborn as sns
import pandas as pd
import numpy as np
from pylab import rcParams
import tensorflow as tf
import tensorflow.keras.backend as K

import tensorflow.keras as keras 

from keras import optimizers, Sequential
from tensorflow.keras.losses import binary_crossentropy, mean_squared_error
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.python.keras import regularizers
from keras.models import Model
from keras.layers import Conv1DTranspose, Dense, LSTM, RepeatVector, TimeDistributed, Input, concatenate, Conv2D, MaxPooling3D, Activation, UpSampling2D, BatchNormalization, add, concatenate, Lambda, Flatten, Conv2DTranspose,DepthwiseConv2D,ConvLSTM2D,TimeDistributed,Dropout, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from scipy import stats

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]


# Prepare Dataset

In [ ]:
!git clone https://github.com/bartkowiaktomasz/har-wisdm-lstm-rnns.git


fatal: destination path 'har-wisdm-lstm-rnns' already exists and is not an empty directory.


In [ ]:
from scipy.io import arff
COLUMN_NAMES = [
    'user',
    'activity',
    'timestamp',
    'x-axis',
    'y-axis',
    'z-axis'
]
LABELS = [
    'Downstairs',
    'Jogging',
    'Sitting',
    'Standing',
    'Upstairs',
    'Walking'
]
dataset=pd.read_csv("/content/har-wisdm-lstm-rnns/data/WISDM_ar_v1.1_raw.txt",header=None,names=COLUMN_NAMES )

In [ ]:
dataset['z-axis'].replace(to_replace=r';',value=r'',regex=True,inplace=True)
dataset['z-axis'] = dataset['z-axis'].astype('float')
dataset.dropna(axis=0, how='any', inplace=True)
dataset.sample(100)

In [ ]:
dataset[['timestamp','x-axis','y-axis','z-axis','activity']]


In [ ]:
dataset[['timestamp','x-axis','y-axis','z-axis','activity']]


In [ ]:
enc = LabelEncoder()
labeled=enc.fit_transform(dataset.loc[:,('activity')].ravel())
dataset['labeled_activites']=labeled
dataset

In [ ]:
mag=np.sqrt(dataset['x-axis']**2+dataset['y-axis']**2+dataset['z-axis']**2)
dataset['magnitude']=mag
dataset

In [ ]:
x=dataset.iloc[:,3:6]
#digitizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')
#x = digitizer.fit_transform(x)

x=(x-x.mean())/x.std()
x=pd.DataFrame(x,columns={'x-axis','y-axis','z-axis'})
x

In [ ]:
dataset['x-axis'] = x['x-axis']
dataset['y-axis'] = x['y-axis']
dataset['z-axis'] = x['z-axis']


# Train-Test split

In [ ]:
train=dataset[dataset['user']<=28]
test=dataset[dataset['user']>28]
train

# Sliding window on Data

In [ ]:
def sliding_window(dataframe,time_steps,steps):
  xs=[]
  ys=[]
  for i in range(0,len(dataframe)-time_steps,steps):
    ext_step=i+time_steps
    if ext_step>len(dataframe)-1:
      break
    xi=dataframe['x-axis'].values[i:i+time_steps]
    y1=dataframe['y-axis'].values[i:i+time_steps]
    z1=dataframe['z-axis'].values[i:i+time_steps]
    yi=stats.mode(dataframe['labeled_activites'][i:i+time_steps])[0][0]
    xs.append([xi,y1,z1])
    ys.append(yi)
  return np.asarray(xs),np.asarray(ys)

In [ ]:
x_train,y_train=sliding_window(train,90,10)
x_test,y_test=sliding_window(test,90,10)

In [ ]:
x_train.shape

In [ ]:
x_train=x_train.reshape(-1,90,3).astype(np.float32)
x_test=x_test.reshape(-1,90,3).astype(np.float32)

In [ ]:
x_train.shape

# CNN Architecture

In [ ]:
def create_model():
  model=keras.models.Sequential([tf.keras.layers.Input(shape=[90,3]),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(128,5,activation ='relu',padding="same"),
    #tf.keras.layers.MaxPool1D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(128,5,activation ='relu',padding="same"),
    tf.keras.layers.MaxPool1D(),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256,5,activation ='relu',padding="same"),
    #tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.35),
    tf.keras.layers.Conv1D(256,5,activation ='relu',padding="same"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
   tf.keras.layers.Dense(12),
    tf.keras.layers.Dense(6,activation="softmax")

                               ])
  return model

# Convolutional Autoencoder Architecture

In [ ]:
def create_model():
  model=keras.models.Sequential([tf.keras.layers.Input(shape=[90,3]),
                                 
    tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding="same", strides=2, activation="relu"),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv1D(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
    tf.keras.layers.Conv1DTranspose(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1DTranspose(filters=64, kernel_size=7, padding="same", strides=2, activation="relu"),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation="softmax")

                               ])
  return model

In [ ]:
model=create_model()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

In [ ]:
Y_train=keras.utils.to_categorical(y_train,num_classes=6)
Y_test=keras.utils.to_categorical(y_test,num_classes=6)

In [ ]:
checkpoint=keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
early_stopping=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=1, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

reduce_LOP=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=0, min_lr=0.001)
Callbacks=[checkpoint,reduce_LOP]

# Training

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)
history=model.fit(x_train,Y_train,batch_size=512,epochs=100,verbose=1,validation_data=(x_test,Y_test),callbacks=Callbacks)


# Plot Results

In [ ]:
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='valid-accuracy')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("accuracy")

In [ ]:
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='valid-loss')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
from keras.models import load_model

model = load_model('best_model.93-0.36.h5')
pred_train=model.predict(x_train)
y_pred_train=np.argmax(pred_train,axis=1)

In [ ]:
    # CONFUSION MATRIX
    confusion_matrix = metrics.confusion_matrix(y_train,y_pred_train)

    plt.figure(figsize=(16, 14))
    sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True);
    plt.title("Confusion matrix on Train Data")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.show();

In [ ]:
    # CONFUSION MATRIX
    confusion_matrix = metrics.confusion_matrix(y_train,y_pred_train)

    plt.figure(figsize=(16, 14))
    sns.heatmap(confusion_matrix/(np.sum(confusion_matrix, axis=1, keepdims=1)), xticklabels=LABELS, yticklabels=LABELS, annot=True);
    plt.title("Confusion matrix on Train Data (with normalization)")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.show();

In [ ]:
from keras.models import load_model

model = load_model('best_model.93-0.36.h5')
pred_test=model.predict(x_test)
y_pred_test=np.argmax(pred_test,axis=1)

In [ ]:
    # CONFUSION MATRIX
    confusion_matrix = metrics.confusion_matrix(y_test,y_pred_test)

    plt.figure(figsize=(16, 14))
    sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True);
    plt.title("Confusion matrix on Test data")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.show();

In [ ]:
    # CONFUSION MATRIX
    confusion_matrix = metrics.confusion_matrix(y_test,y_pred_test)

    plt.figure(figsize=(16, 14))
    sns.heatmap(confusion_matrix/(np.sum(confusion_matrix, axis=1, keepdims=1)), xticklabels=LABELS, yticklabels=LABELS, annot=True);
    plt.title("Confusion matrix on Test Data (with normalization)")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.show();